# Data Augmentation

In [ ]:
!unzip modified.zip

In [ ]:
import os

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

n_images = 15

In [ ]:
X = []
y = []

for i in range(-5, 6):
  image_batch = []
  for ts in range(1, 3):
    filename = f'{i}_{ts}_unmasked.jpg'
    if not os.path.exists(filename): continue
    fig = cv2.imread(filename)
    #fig = fig[200:800, 200:800] # Center 600px * 600px
    fig = cv2.resize(fig, (250, 250))
    image_batch.append(cv2.cvtColor(fig, cv2.COLOR_BGR2GRAY).reshape(250, 250, 1))
  image_batch = np.array(image_batch)
  data_generator = ImageDataGenerator(
    horizontal_flip=False, vertical_flip=False, rotation_range=180,
    fill_mode='constant', zoom_range=[0.75,1.25], brightness_range=(0.8,1.2),
    channel_shift_range=20
  )
  data_generator.fit(image_batch)

  data_gen_iter = data_generator.flow(image_batch)
  X.append(image_batch)
  y += [i] * image_batch.shape[0]

  for i in range(n_images):
    aug_image_batch = next(data_gen_iter)
    aug_image = aug_image_batch.reshape(-1, 250, 250, 1)
    aug_image = aug_image.astype('int')
    X.append(aug_image)
    y += [i] * aug_image.shape[0]

In [ ]:
total = sum(item.shape[0] for item in X)
total

336

In [ ]:
images = np.empty((total, 250, 250, 1))
curr_idx = 0

for i in range(len(X)):
  images[curr_idx:curr_idx+X[i].shape[0]] = X[i]
  curr_idx += X[i].shape[0]

In [ ]:
X = images
del images
X.shape

(336, 250, 250, 1)

In [ ]:
import gc
gc.collect()

10081

# ResNet50 Model

In [ ]:
import numpy as np
import cv2
import gc

# X = np.load('content/data/X.npy')
# y = np.load('content/data/Y.npy')
# X_shrinked = []
# for i in X:
#   X_shrinked.append(cv2.resize(i, (150, 150)))

# X = np.array(X_shrinked)
# del X_shrinked
# gc.collect()

In [ ]:
X[1].shape

(250, 250, 1)

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add

def residual_block(input_tensor, kernel_size, num_filters, name):
  x = Conv2D(kernel_size=(1, 1), filters = num_filters // 4, kernel_initializer = 'he_normal', name=f'Conv_{name}_a')(input_tensor)
  x = BatchNormalization(name=f'BatchNorm_{name}_a')(x)
  x = Activation('relu')(x)

  x = Conv2D(kernel_size=kernel_size, padding='same', filters=num_filters//4, kernel_initializer = 'he_normal', name=f'Conv_{name}_b')(x)
  x = BatchNormalization(name=f'BatchNorm_{name}_b')(x)
  x = Activation('relu')(x)

  x = Conv2D(kernel_size=(1, 1), filters = num_filters, kernel_initializer = 'he_normal', name=f'Conv_{name}_c')(x)
  x = BatchNormalization(name=f'BatchNorm_{name}_c')(x)
  x = Add()([input_tensor, x])

  x = Activation('relu')(x)

  return x

In [ ]:
def downsize_block(input_tensor, kernel_size, strides, num_filters, name):
  x = Conv2D(kernel_size=(1,1), filters=num_filters//4, strides=strides, kernel_initializer='he_normal', name=f'Conv_{name}_a')(input_tensor)
  x = BatchNormalization(name=f'BatchNorm_{name}_a')(x)
  x = Activation('relu')(x)

  x = Conv2D(kernel_size=kernel_size, filters=num_filters//4, padding='same', kernel_initializer='he_normal', name=f'Conv_{name}_b')(x)
  x = BatchNormalization(name=f'BatchNorm_{name}_b')(x)
  x = Activation('relu')(x)

  x = Conv2D(kernel_size=(1,1), filters=num_filters, kernel_initializer='he_normal', name=f'Conv_{name}_c')(x)
  x = BatchNormalization(name=f'BatchNorm_{name}_c')(x)

  shortcut = Conv2D(kernel_size=(1,1), filters=num_filters, strides=strides, kernel_initializer='he_normal', name=f'Shortcut_{name}')(input_tensor)
  shortcut = BatchNormalization(name=f'Shortcut_BatchNorm_{name}')(shortcut)

  x = Add()([shortcut, x])
  output_tensor = Activation('relu')(x)

  return output_tensor

In [ ]:
from tensorflow.keras.layers import ZeroPadding2D

def first_convolution(input_tensor):
  x = ZeroPadding2D(padding=(3,3))(input_tensor)
  x = Conv2D(filters=64, strides=(2,2), kernel_size=(7,7), kernel_initializer='he_normal')(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation('relu')(x)

  return x

In [ ]:
from tensorflow.keras.layers import MaxPool2D

def first_pooling(input_tensor):
  x = ZeroPadding2D(padding=(1,1))(input_tensor)
  x = MaxPool2D(pool_size=(3,3), strides=(2,2))(x)

  return x

In [ ]:
from tensorflow.keras.layers import Input, GlobalMaxPooling2D, Dense
from tensorflow.keras.models import Model
input_tensor = Input(shape=(250, 250, 1))

x = first_convolution(input_tensor)

x = first_pooling(x)
x = downsize_block(x, (3,3), (1,1), 256, 's2d')
x = residual_block(x, (3,3), 256, 's2r1')
x = residual_block(x, (3,3), 256, 's2r2')

x = downsize_block(x, (3, 3), (2, 2), 512, 's3d')
x = residual_block(x, (3, 3), 512, 's3r1')
x = residual_block(x, (3, 3), 512, 's3r2')
x = residual_block(x, (3, 3), 512, 's3r3')

x = downsize_block(x, (3, 3), (2, 2), 1024, 's4d')
x = residual_block(x, (3, 3), 1024, 's4r1')
x = residual_block(x, (3, 3), 1024, 's4r2')
x = residual_block(x, (3, 3), 1024, 's4r3')
x = residual_block(x, (3, 3), 1024, 's4r4')
x = residual_block(x, (3, 3), 1024, 's4r5')

x = downsize_block(x, (3, 3), (2, 2), 2048, 's5d')
x = residual_block(x, (3, 3), 2048, 's5d1')
x = residual_block(x, (3, 3), 2048, 's5d2')

x = GlobalMaxPooling2D()(x)
x = Dense(1)(x)

model = Model(inputs=input_tensor, outputs=x)

In [ ]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 250, 250, 1)]        0         []                            
                                                                                                  
 zero_padding2d_10 (ZeroPad  (None, 256, 256, 1)          0         ['input_6[0][0]']             
 ding2D)                                                                                          
                                                                                                  
 conv2d_14 (Conv2D)          (None, 125, 125, 64)         3200      ['zero_padding2d_10[0][0]']   
                                                                                                  
 batch_normalization_14 (Ba  (None, 125, 125, 64)         256       ['conv2d_14[0][0]']     

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
model_checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-5), loss='mse', metrics=['mae', 'mse'])

In [ ]:
X = X / 255
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=[X_test, y_test], callbacks=[early_stopping, reduce_lr, model_checkpoint])

Epoch 1/100
10/10 [==============================] - 32s 611ms/step - loss: 42.0324 - mae: 5.2577 - mse: 42.0324 - val_loss: 2388.8779 - val_mae: 43.9285 - val_mse: 2388.8779 - lr: 1.0000e-05
Epoch 2/100
10/10 [==============================] - 5s 495ms/step - loss: 28.7105 - mae: 4.2305 - mse: 28.7105 - val_loss: 153.1214 - val_mae: 11.0813 - val_mse: 153.1214 - lr: 1.0000e-05
Epoch 3/100
10/10 [==============================] - 5s 499ms/step - loss: 20.6247 - mae: 3.7069 - mse: 20.6247 - val_loss: 102.3755 - val_mae: 8.7109 - val_mse: 102.3755 - lr: 1.0000e-05
Epoch 4/100
10/10 [==============================] - 4s 387ms/step - loss: 19.3781 - mae: 3.5278 - mse: 19.3781 - val_loss: 138.6614 - val_mae: 10.4480 - val_mse: 138.6614 - lr: 1.0000e-05
Epoch 5/100
10/10 [==============================] - 4s 393ms/step - loss: 15.7725 - mae: 3.0795 - mse: 15.7725 - val_loss: 113.0452 - val_mae: 9.3747 - val_mse: 113.0452 - lr: 1.0000e-05
Epoch 6/100
10/10 [==============================] - E